In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/nlp_project

/content/drive/MyDrive/nlp_project


In [8]:
!pip install pyarabic
!pip install arabic-stopwords
!pip install qalsadi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 6.6 MB/s 
     |████████████████████████████████| 9.2 MB 41.5 MB/s 
     |████████████████████████████████| 138 kB 80.7 MB/s 
     |████████████████████████████████| 251 kB 55.2 MB/s 
     |████████████████████████████████| 51 kB 557 kB/s 
     |████████████████████████████████| 332 kB 62.4 MB/s 
  Created wheel for pickledb: filename=pickleDB-0.9.2-py3-none-any.whl size=4269 sha256=268b3395f8dc1eb927a7f1fc2cc4a824764dbf787ffabb4956000d275314ac33
  Stored in directory: /root/.cache/pip/wheels/88/91/d4/ef2e6a46ad2bc41f9cfad35fa2db5b34357a5e4da67c385ffa
Successfully built pickledb


In [9]:
from sklearn import metrics
import pandas as pd
from torch import nn
from pyarabic.araby import tokenize
import numpy as np
import pickle
import spacy
import torch

# from model_building import Classifier 
from pre_processing_post import processPost
from feature_extraction import get_ngram_features, get_word_embedding_features, avg_word_vector
from gru_model import ArabicDataset, Classifier, evaluate, train

In [10]:
# needed functions
def print_report(y_pred, y_test):
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

In [11]:
# load AraVec Spacy model
nlp = spacy.load("./spacy.aravec.model/")

In [12]:
test_data = pd.read_csv('./DataSet/dev.csv',sep=',',header=0)
test_data['text'] = test_data['text'].apply(lambda x: processPost(x))
test_data['category'] = test_data['category'].astype('category').cat.codes

In [13]:
# load word2index dictionary
with open('./vocab/category/word2index.pickle', 'rb') as f:
    word2index = pickle.load(f)

In [14]:
weights_train_matrix = []
for word in word2index:
  weights_train_matrix.append(nlp(word).vector)

weights_train_matrix = torch.from_numpy(np.array(weights_train_matrix))
weights_train_matrix.size()

torch.Size([12538, 100])

td-idf feature

In [29]:
# load SVC model
with open('./models/category/LinearSVC_tfidf.sav', 'rb') as f:
    SVC_model = pickle.load(f)

with open('./models/category/TFIDFVectorizer.sav', 'rb') as f:
    word_vectorizer = pickle.load(f)

X_test_tfidf = word_vectorizer.transform(test_data['text'])
y_pred = SVC_model.predict_proba(test_data['text'])

In [30]:
with open('./models/category/RandomForest_tfidf.sav', 'rb') as f:
    rf = pickle.load(f)

y_pred += rf.predict_proba(X_test_tfidf)

In [31]:
#load GRU model
with open('./models/category/GRU_Ara2Vec.pth', 'rb') as f:
    gru_model = Classifier(weights_train_matrix, n_classes=10, n_layer=2)
    gru_model.load_state_dict(torch.load(f)) 
    gru_model.eval()

test_data_tokenized = test_data['text'].apply(tokenize)
test_data_tokenized_as_num = test_data_tokenized.apply(lambda x: [word2index[word] for word in x if word in word2index])
test_dataset = ArabicDataset(list(test_data_tokenized_as_num), test_data['category'], word2index['<فراغ>'])
y_pred += evaluate(gru_model,test_dataset)

100%|██████████| 32/32 [00:00<00:00, 56.86it/s]


accuracy: 0.492
              precision    recall  f1-score   support

           0       0.29      0.40      0.33        10
           1       0.75      0.83      0.78       145
           2       0.76      0.41      0.53       545
           3       0.12      0.18      0.14        17
           4       0.42      0.60      0.50       128
           5       0.20      0.50      0.29        82
           6       0.14      0.30      0.19        20
           7       0.33      0.50      0.40         2
           8       0.06      0.07      0.06        15
           9       0.30      0.47      0.37        36

    accuracy                           0.49      1000
   macro avg       0.34      0.43      0.36      1000
weighted avg       0.61      0.49      0.51      1000


Test Accuracy: 0.492000013589859


In [32]:
y_pred /= 3
y_pred = np.argmax(y_pred, axis=1)
print_report(y_pred, test_data['category'])

              precision    recall  f1-score   support

           0       0.27      0.30      0.29        10
           1       0.80      0.84      0.82       145
           2       0.74      0.73      0.74       545
           3       0.08      0.06      0.07        17
           4       0.47      0.58      0.52       128
           5       0.23      0.07      0.11        82
           6       0.13      0.20      0.16        20
           7       0.50      0.50      0.50         2
           8       0.08      0.07      0.07        15
           9       0.30      0.47      0.37        36

    accuracy                           0.63      1000
   macro avg       0.36      0.38      0.36      1000
weighted avg       0.62      0.63      0.62      1000

accuracy: 0.628
